## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
from pymystem3 import Mystem # библиотека с функцией лемматизации
from collections import Counter # контейнер для подсчета числа лемматизированных слов 
import re # доп модуль для работы с регулярными выражениями (понадобится для нумерации типов целей кредита после лемматизации)
clients = pd.read_csv('/datasets/data.csv')
clients.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
clients.tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


In [3]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Каждая строка таблицы содержит информацию об определенном клиенте и общую информацию о нем: возраст, общий трудовой стаж, наличие образования, семейное положение. После получения общей информации о таблице, сразу была выявлена проблема с пропусками в столбцах days_employed и total_income, которую необходимо решить.

In [4]:
clients.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В данных присутствуют пустые значения в столбцах days_employed и total_income. Можно предположить, что в строках, где нет информации о трудовом стаже, также нет и информации о доходе.

In [5]:
clients[(clients['days_employed'].isnull() == True) & (clients['days_employed'].isnull() == True)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
children            2174 non-null int64
days_employed       0 non-null float64
dob_years           2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
gender              2174 non-null object
income_type         2174 non-null object
debt                2174 non-null int64
total_income        0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


Гипотеза подтвердилась. В строках, где нет информации о трудовом стаже, также нет и информации о доходе. Пустые значения свидетельствуют, что при отсутствии информации о трудовом стаже сложно определить данные по доходу.

In [6]:
clients.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

На мой взгляд, не совсем корректно указаны названия столбцов. Переименуем названия столбцов, чтобы лучше ориентироваться в них.

In [7]:
clients.set_axis(['children', 'days_employed', 'age', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt_credit',
       'income_month', 'purpose'], axis='columns', inplace=True) # изменение названий столбцов
clients.head(10)

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt_credit,income_month,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Проанализируем каждый столбец таблицы с целью корректировки и выявления "аномалий".

In [8]:
clients['children'].value_counts().sort_index() # найдем количество уникальных значений столбца про количетво детей

-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

**Минимальное количество детей** - отрицательное. Всего таких 47 строк - небольшое количество к общему объему данных. <br> 
**Максимальное количество детей** - 20, вызывает сомнения, возможно технологическая ошибка. Всего таких клиентов: 76.

In [9]:
clients[clients['days_employed'] <= 0].head() # найдем клиентов, у которых указан отрицательный трудовой стаж 

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt_credit,income_month,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья


Отрицательные минимальные значения в столбце о трудовом стаже - возможно технологическая ошибка, берем по модулю.

In [10]:
len(clients[clients['age'] <= 0]) # найдем количество клиентов, у которых указан возраст меньше нуля

101

Выявили, что у нас есть клиенты, возраст которых не указан. Данный факт не полияет на ответы на основные вопросы заказчика, поэтому оставим как есть.

In [11]:
clients['education'].value_counts() # определим, какие степени образования есть в данных

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Были выявлены дубликаты - исправим, приведем все данные к нижнему регистру.

In [12]:
clients['family_status'].value_counts()
# по семейному положению дубликатов нет

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [13]:
clients['gender'].value_counts() # посмотрим, встречаются ли только мужской и женский пол

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [14]:
clients[clients['gender'] == 'XNA']

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt_credit,income_month,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Найден 1 неизвестный вид пола **"XNA"**. Такую аномалию следует удалить, т.к. она встречается всего 1 раз, это никак не отразиться на датасете и результатах исследования.

In [15]:
clients.groupby('income_type')['age'].count() # посмотрим, какие есть типы занятости

income_type
безработный            2
в декрете              1
госслужащий         1459
компаньон           5085
пенсионер           3856
предприниматель        2
сотрудник          11119
студент                1
Name: age, dtype: int64

In [16]:
clients[clients['income_month'] == 0]

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt_credit,income_month,purpose


В наших данных нет клиентов с нулевым уровнем дохода

In [17]:
clients['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Цель кредита прописана везде по каждому клиенту, это важно, чтобы позже определить "как разные цели влияют на возврат кредита в срок".

### Вывод

1 - После получения общей информации о таблице, сразу была выявлена проблема с пропусками в столбцах days_employed и total_income, которую необходимо решить. Так как у нас нет возможности уточнить данные по каждому из клиентов, воспользуемся медианными значениями. Для этого можно посчитать медианные значения по трудовому стажу и доходу для каждого типа занятости. После заменить пропущенные значения в этих столбцах на медианные значения <br> 
2 - Отрицательные значения в столбце о трудовом стаже - возможно технологическая ошибка, берем по модулю. <br> 
3 - Значения, указанные в трудовом стаже и доходе клиентов имеют вещественный тип данных. Необходимо перевести значения в целочисленный тип (int). <br> 
4 - Минимальное количество детей - отрицательное. Всего таких 47 строк - небольшое количество к общему объему данных. <br> Максимальное количество детей - 20, вызывает сомнения, возможно технологическая ошибка. Всего таких клиентов: 76. Можно предположить, что данные значения (-1 и 20) появились вследствии некорректного заполнения анкет сотрудниками кредитного отдела (вместо 1 появились отрицательные значения). Так как у нас нет возможности уточнить, в чем причина такого заполнения, отрицательное значение (-1) заменим на модуль числа: 1, а значение 20 трогать не будем, отнесем его к многодетным семьям (необходимо при категоризации). <br> 
5 - Есть клиенты, возраст которых указан меньше нуля - трогать не будем, т.к. данные значения не повлияют на ответы на основные вопросы заказчика. <br> 
6 - В столбце о типах образования есть дубликаты (необходимо привести к нижнему регистру). <br> 
7- Найден 1 неизвестный вид пола "XNA" - 1. Значение XNA вероятно появилось в результате технологической ошибки при копировании или смене формата. Такую аномалию следует удалить, т.к. она встречается всего 1 раз, это никак не отразиться на датасете и результатах исследования.

### Шаг 2. Предобработка данных

### Обработка пропусков

Замена пропусков в столбцах days_employed и total_income.
Создадим сводную таблицу (медианные значения для трудового стажа и дохода для каждого типа занятости)

In [18]:
clients_pivot = clients.pivot_table(index = ['income_type'], values = ['days_employed', 'income_month'], aggfunc = 'median')
clients_pivot

,days_employed,income_month
income_type,,
безработный,366413.652744,131339.751676
в декрете,-3296.759962,53829.130729
госслужащий,-2689.368353,150447.935283
компаньон,-1547.382223,172357.950966
пенсионер,365213.306266,118514.486412
предприниматель,-520.848083,499163.144947
сотрудник,-1574.202821,142594.396847
студент,-578.751554,98201.625314


Напишем 2 функции, которые будут обрабатывать: 1 - каждую строку таблицы clients и проверять, если по доходу клиента нет данных, заменять это значение дохода по каждому типу занятости на полученное медианное значение из сводной таблицы clients_pivot, 2 - каждую строку таблицы clients и проверять, если по трудовому стажу клиента нет данных, заменять это значение на полученное медианное значение трудового стажа по каждому типу занятости из сводной таблицы clients_pivot.

In [19]:
def median_replacement_income(row):
    income_month = row['income_month'] # доход
    income_type = row['income_type'] # тип занятости
    if not pd.isnull(income_month): # если значение не "пустое", то функция вернет это же значение 
        return income_month
    else:
        return clients_pivot.loc[income_type]['income_month'] # в противном случае (если есть пропуск) - замена на медианное значение

def median_replacement_days(row):
    days_employed = row['days_employed'] # стаж
    income_type = row['income_type'] # тип занятости
    if not pd.isnull(days_employed):
        return days_employed
    else:
        return clients_pivot.loc[income_type]['days_employed']

In [20]:
clients['income_month'] = clients.apply(median_replacement_income, axis=1) 
# заполнем наш столбец о доходе клиентов обработанными значенями-результатами работы функции median_replacement_income с помощью метода apply

In [21]:
clients['days_employed'] = clients.apply(median_replacement_days, axis=1)
# заполнем наш столбец о трудовом стаже обработанными значенями-результатами работы функции median_replacement_income с помощью метода apply

Отрицательные значения в столбце о трудовом стаже - берем по модулю.

In [22]:
def to_positive(value):
    if value <0: # если значение отрицательное, берем по модулю (умножаем на (-1))
        value *= -1
        return value 
    else:
        return value # если число положительное, возвращаем это же значение 
clients['days_employed'] = clients['days_employed'].apply(to_positive)
# функция, перебирающая каждое значение (если отрицательное, умножаем на (-1), чтобы получить число > 0)
# примением функцию к столбцу о трудовом стаже

Некорректные данные о количестве детей: заменяем отрицательное значение (-1) на положительное (1).

In [23]:
clients['children'] = clients['children'].replace(-1, 1)

Найден 1 неизвестный вид пола "XNA" - 1. Значение XNA вероятно появилось в результате технологической ошибки при копировании или смене формата. Такую аномалию следует удалить, т.к. она встречается всего 1 раз, это никак не отразиться на датасете и результатах исследования.

In [24]:
clients.drop(clients.index[10701], inplace=True)

### Вывод

Пропущенные значения в строках о трудовом стаже и доходе клиентов заменили на медианное значение по типу занятости. Отрицательные значения в трудовом стаже заменили на положительные. Изменили некорректные значения по количеству детей. Удалили строку с неизвестным видом пола - не отразиться на результатах.

### Замена типа данных

In [25]:
clients['days_employed'] = clients['days_employed'].astype('int')
clients['income_month'] = clients['income_month'].astype('int')
# перевод значений о трудовом стаде и доходе в целочиленный тип

### Вывод

Заменили значения о трудовом стаже и доходе на целочисленный тип данных.

### Обработка дубликатов

In [26]:
clients['education'] = clients['education'].str.lower()
clients['education'].value_counts() # проверка, теперь данные содержат только 5 типов образования

среднее                15233
высшее                  5260
неоконченное высшее      743
начальное                282
ученая степень             6
Name: education, dtype: int64

In [27]:
clients.duplicated().sum() # определим , сколько дубликатов в данных

71

Причина возникновения дубликатов - может быть задвоение данных. Избавимся от дубликатов методом drop_duplicates()

In [28]:
clients = clients.drop_duplicates()
clients.duplicated().sum() # проверим, что дубликаты удалены

0

### Вывод

Избавились от дубликатов, которые могли появиться вследствие задвоения данных. Также привели все данные в столбце об образовании к нижнему регистру.

### Лемматизация

В столбце о целях кредита есть похожие по смыслу друг на друга цели (операции с недвижимостью, покупка недвижимости). Необходимо их лемматизировать, чтобы выяснить количество уникальных лемм. Этот шаг важен, т.к. нужен для ответа на один из основных вопросов заказзика о зависимости целей кредита и возврата кредита в срок.

In [29]:
m = Mystem() 
def lemm(purpose): # функция для лемматизации
    lemmas = ''.join(m.lemmatize(purpose))
    return lemmas

In [30]:
clients['purpose_lemm'] = clients['purpose'].apply(lemm) # добавляем новый столбец с лемматизацией 
clients.head(3)

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt_credit,income_month,purpose,purpose_lemm
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье\n
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль\n
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье\n


In [31]:
Counter(clients['purpose_lemm']) # подсчет количества упоминаний

Counter({'покупка жилье\n': 646,
         'приобретение автомобиль\n': 461,
         'дополнительный образование\n': 460,
         'сыграть свадьба\n': 765,
         'операция с жилье\n': 652,
         'образование\n': 447,
         'на проведение свадьба\n': 768,
         'покупка жилье для семья\n': 638,
         'покупка недвижимость\n': 620,
         'покупка коммерческий недвижимость\n': 661,
         'покупка жилой недвижимость\n': 606,
         'строительство собственный недвижимость\n': 635,
         'недвижимость\n': 633,
         'строительство недвижимость\n': 619,
         'на покупка подержать автомобиль\n': 478,
         'на покупка свой автомобиль\n': 505,
         'операция с коммерческий недвижимость\n': 650,
         'строительство жилой недвижимость\n': 624,
         'жилье\n': 646,
         'операция со свой недвижимость\n': 627,
         'автомобиль\n': 972,
         'заниматься образование\n': 408,
         'сделка с подержанный автомобиль\n': 486,
         'получ

In [32]:
category = ['жилье', 'недвижимость', 'образование', 'свадьба', 'автомобиль'] 
numbers = [1, 2, 3, 4, 5] 
new_dict = {category: numbers for category, numbers in zip(category, numbers)}
new_dict # создаем словарь из 5 основных категорий целей кредита

{'жилье': 1,
 'недвижимость': 2,
 'образование': 3,
 'свадьба': 4,
 'автомобиль': 5}

In [33]:
def purpose_category(purpose): 
    for key in new_dict:
        if re.search(key, purpose): # ищет по всей строке по заданному шаблону (key в словаре), возвращает первое найденное выражение
            return new_dict[key] # возвращает значение ключа, т.е. номер цели кредита 
# можно написать другую функцию, которая будет работать так: если в clients['purpose_lemm'] недвижимость, верни недвижимость

In [34]:
clients['key'] = clients['purpose_lemm'].apply(purpose_category) # создаем новый столбец с типами целей кредита согласно данным из словаря

In [35]:
clients.head(3) # проверка результата 

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt_credit,income_month,purpose,purpose_lemm,key
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье\n,1
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль\n,5
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье\n,1


### Вывод

Выделили леммы по столбцу о целях кредита. Присвоили каждой лемме свой номер (понадобится для нахождения взаимосвязи цели и возврата кредита в срок).

### Категоризация данных

Распределение клиентов по количеству детей. Если у клиента нет детей попадает в категорию: "детей нет", если 1-2 ребенка - категория "1-2 ребенка", если больше детей - "многодетные"

In [36]:
def children_category(children):
    if children == 0:
        return 'детей нет'
    elif children >= 1 and children <= 2:
        return '1-2 ребенка'
    else:
        return 'многодетные'

Распределение клиентов по семейному положению.

In [37]:
def family_status_category(status):
    if status == 'женат / замужем':
        return 'в браке'
    if status == 'гражданский брак':
        return 'гражданский брак'
    if status == 'Не женат / не замужем':
        return 'не в браке'
    if status == 'в разводе':
        return 'в разводе'
    else:
        return 'вдовец / вдова'

Распределение клиентов по доходу. Если доход клиента менее 50000, он попадает в категорию низкого дохода, в диапазоне между 50000 и 150000 - средний доход, диапозон 150000 - 400000 - выше среднего, большее 400000 - высокий.

In [38]:
clients['income_month'].max()

2265604

In [39]:
def income_month_category(income):
    if income < 50000:
        return 'низкий'
    elif income >= 50000 and income <= 150000:
        return 'средний'
    elif income > 150000 and income <= 400000:
        return 'выше среднего'
    else:
        return 'высокий'

In [40]:
# с помощью метода apply применим написанные функции и добавим новые столбцы для дальнейшей работы 
clients['children_category'] = clients['children'].apply(children_category)
clients['fam_status_category'] = clients['family_status'].apply(family_status_category)
clients['income_category'] = clients['income_month'].apply(income_month_category)

In [41]:
clients.head()

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt_credit,income_month,purpose,purpose_lemm,key,children_category,fam_status_category,income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье\n,1,1-2 ребенка,в браке,выше среднего
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль\n,5,1-2 ребенка,в браке,средний
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье\n,1,детей нет,в браке,средний
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование\n,3,многодетные,в браке,выше среднего
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба\n,4,детей нет,гражданский брак,выше среднего


### Вывод

Проверили категоризацию пользователей по различным критериям: по количеству детей, семейному положению и уровню дохода.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [42]:
clients.pivot_table(index = ['children_category'], values = ['debt_credit']).sort_values(by='debt_credit')

,debt_credit
children_category,
детей нет,0.075444
многодетные,0.085526
1-2 ребенка,0.092515


### Вывод

Клиенты, у которых нет детей, менее склонны к невыплате кредита в срок (это можно обосновать тем, что у них нет дополнительной статьи расхода на содержание детей). Интересно, что клиенты, у которых по 1-2 ребенка чаще не выполняют свои кредитные обязательства, чем многодетные семьи (клиенты, у которых от 3 и более детей). 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [43]:
clients.pivot_table(index = ['fam_status_category'], values = ['debt_credit']).sort_values(by='debt_credit')

,debt_credit
fam_status_category,
вдовец / вдова,0.065693
в разводе,0.071130
в браке,0.075452
гражданский брак,0.093494
не в браке,0.097509


### Вывод

Клиенты, находящиеся в браке, реже имеют просрочки по кредитам (возможно, потому что имеют общий бюджет с супругом/супругой и лучше контролируют свой бюджет), чем клиенты, которые в браке не состоят. Клиенты, находящиеся в гражданском браке, также имеют высокий показатель невыплаты кредита в срок.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [44]:
clients.pivot_table(index = ['income_category'], values = ['debt_credit']).sort_values(by='debt_credit')

,debt_credit
income_category,
высокий,0.058601
низкий,0.061828
выше среднего,0.078701
средний,0.084881


### Вывод

Клиенты со средним уровнем дохода (от 50000 до 150000) имеют больше невыплат кредита, чем люди с низким уровнем дохода.

- Как разные цели кредита влияют на его возврат в срок?

In [45]:
clients.pivot_table(index = ['key'], values = ['debt_credit']).sort_values(by='debt_credit')

,debt_credit
key,
1,0.069058
2,0.074646
4,0.080034
3,0.092200
5,0.093590


### Вывод

Напомним, что каждая цель кредита имеет свой номер (жилье - 1, недвижимость - 2, образование - 3, свадьба -4, автомобиль - 5). Больше всего невыплат по кредиту у клиентов, которые берут кредит на автомобиль. Клиенты, которые берут кредиты на жилье и недвижимость реже имеют задолженности по возврату кредита.

### Шаг 4. Общий вывод

В результате работы были выявлены следующие инсайты: <br> 
1 - **Зависимость между наличием детей и возвратом кредита** в срок существует. Клиенты, у которых нет детей, менее склонны к невыплате кредита в срок. Интересно, что клиенты, у которых по 1-2 ребенка чаще не выполняют свои кредитные обязательства, чем многодетные семьи (клиенты, у которых от 3 и более детей). <br> 
2 - **Зависимость между семейным положением и возвратом кредита в срок** следующая: клиенты, находящиеся в браке, реже имеют просрочки по кредитам, чем клиенты, не состоящие в законных отношениях. <br> 
3 - **Зависимость между уровнем дохода и возвратом кредита в срок**. Клиенты со средним уровнем дохода (от 50000 до 150000) имеют больше невыплат кредита, чем люди с низким уровнем дохода (менее 50000). <br> 
4 - **Влияние разных целей кредита на его возврат**. Больше всего невыплат по кредиту у клиентов, которые берут кредит на автомобиль. Клиенты, которые берут кредиты на жилье и недвижимость реже имеют задолженности по возврату кредита.